In [ ]:
from google.colab import drive
import pandas as pd
import numpy
drive.mount('/content/drive')
songs = pd.read_csv('/content/drive/My Drive/Colab Notebooks/yataros1.csv',header=None)

dataset_sec_rec=songs.copy()
dataset_sec_rec=dataset_sec_rec.transpose()

dataset_sec_rec.drop(0, axis=1, inplace = True)
dataset_sec_rec = dataset_sec_rec.iloc[1:]
dataset_sec_rec = dataset_sec_rec.reset_index(drop=True)
dataset_sec_rec = dataset_sec_rec.to_numpy()
dataset_sec_rec=dataset_sec_rec.astype(numpy.float64)

# songs=songs.transpose()
# print(songs.isnull().sum())

songs.drop(0, axis=1, inplace = True)
songs = songs.iloc[1:]
songs = songs.reset_index(drop=True)
dataset = songs.to_numpy()

# dataset=dataset[0:10,0:12]
dataset=dataset.astype(numpy.float64)
n=numpy.size(dataset,axis=0)
# print(numpy.size(dataset,axis=0),numpy.size(dataset,axis=1))
# print(dataset)
cents_graph_global=[]
# print(n)

Mounted at /content/drive


In [ ]:
##CUPY
from timeit import default_timer
import math
import cupy as cp
import time
def calculate_distance_matrix(X):
    # X es una matriz en CuPy que contiene tus datos de entrada

    # Calcular las diferencias entre los puntos en paralelo
    diff = X[:, None, :] - X[None, :, :]

    # Elevar al cuadrado las diferencias y sumar a lo largo de las características
    squared_diff = cp.sum(diff**2, axis=2)

    # Calcular la matriz de distancias tomando la raíz cuadrada de las diferencias
    distance_matrix = cp.sqrt(squared_diff)
    # print(distance_matrix)
    return distance_matrix
    # return squared_diff

KCentresGPU = cp.RawKernel(r'''
extern "C" __global__
void KCentresGPU(const float *D_d,int* K_d,signed int* L_d,int N,int K) {
    int i=threadIdx.x+blockIdx.x*blockDim.x;
    float minimum=float('inf');
    int pos=-1;
    int kk,m;
    if (i<N){
      for(int k = 0; k < K; k++) {
        kk=K_d[k];
        m=i*N+kk;

        if (D_d[m]<minimum){
          minimum=D_d[m];
          pos=k;
        }
      }
      L_d[i]=pos;
    }
}
''', 'KCentresGPU')

findMaximaGPU = cp.RawKernel(r'''
extern "C" __global__
void findMaximaGPU(const float *D_d,int *L_d,float *M_d,int N)
{
int i=threadIdx.x+blockIdx.x*blockDim.x;
float maximum=-1.0;

int m1,m2;

if (i<N){
  int cluster_label=L_d[i];
  for(int j = 0; j < N; j++) {
    if(cluster_label==L_d[j]){
      m1=i*N+j;

      if(D_d[m1]>maximum){
        maximum=D_d[m1];
      }
    }
  }
  m2=cluster_label*N+i;
  M_d[m2]=maximum;
}

}
''', 'findMaximaGPU')

findMinimumOfMaximaGPU = cp.RawKernel(r'''
extern "C" __global__
void findMinimumOfMaximaGPU(int *t_d,float *M_d,int N,int K)
{
  int k=threadIdx.x+blockIdx.x*blockDim.x;
  float minimum=float('inf');
  int m3,pos;

  if(k<K){
    for(int j=0;j<N;j++){
      m3=k*N+j;
      if(M_d[m3]<minimum){
        minimum=M_d[m3];
        pos=j;
      }
    }
    t_d[k]=pos;
  }
}
''', 'findMinimumOfMaximaGPU')



def k_centres(distance_matrix,N,k_length,L_d,M,t,nrep,block1,grid1):
  elapsed_time=0
  dopt=cp.inf


  for tri in range(nrep):
    # K=cp.random.permutation(N).astype(cp.int32)
    # K=K[0:k_length]
    K=cp.array([3, 14, 35, 24, 39], dtype=cp.int32)

    start_time = time.time()
    while True:
      # Hallar los elementos más cercanos a los centroides
      KCentresGPU((block1[0],), (grid1[0],), (distance_matrix,K,L_d,N,k_length))

      #Hallar la distancia mayor de cada elemento para cada cluster
      findMaximaGPU((block1[0],), (grid1[0],), (distance_matrix,L_d,M,N))

      #Hallar los minimos de cada cluster
      findMinimumOfMaximaGPU((block1[0],), (grid1[0],), (t,M,N,k_length))



      if cp.all(t==K):
        # print(t)
        dmin,labs=cp.min(distance_matrix[t,:],axis=0),cp.argmin(distance_matrix[t,:],axis=0)
        dmin=cp.max(dmin)
        break

      # K=cp.asarray(t)
      K=t.copy()

    finish_time = time.time()
    elapsed_time+=(finish_time-start_time)

    if(dmin<=dopt):
        dopt=dmin
        labels=cp.transpose(labs)
        jopt=t.tolist()
  dm=cp.zeros(k_length)

  for i in range(k_length):
    L=cp.where(labels==i)[0]
    dm[i]=cp.max(distance_matrix[jopt[i],L])

  print("Tiempo de K-centres: ", elapsed_time/nrep)

  return labels,jopt,dm

#dataset
dataset=cp.asarray(dataset_sec_rec,dtype=cp.float32)

#N
N=cp.size(dataset,axis=0)

#K_length
k_length=5

#inicialización L_d
L_d=cp.empty(N,dtype=cp.int32)

#M
M=cp.full((k_length,N),cp.inf)
M = M.astype(cp.float32)

#t
t=cp.full((k_length),-1,dtype=cp.int32)

#nrep
nrep=25



max_threads_per_block=1024
dimx1 = int(math.sqrt(max_threads_per_block));
block1=(dimx1, 1,1);
grid1=(int((N + block1[0] - 1) / block1[0]), 1,1);



inicio=time.time()
# cálculo de la matriz de disimilutes
distance_matrix = calculate_distance_matrix(dataset)
fin=time.time()
print("Tiempo de Matriz de distancias: ", (fin-inicio))


labels,jopt,dm=k_centres(distance_matrix,N,k_length,L_d,M,t,nrep,block1,grid1)


print("\n",dm,labels)
print("\nCENTROIDES :",jopt)

Tiempo de Matriz de distancias:  0.0004837512969970703
Tiempo de K-centres:  0.0016910171508789063

 [1.87372016e+08 9.36759720e+01 5.79448047e+04 1.35219824e+04
 8.05443726e+02] [2 3 3 3 3 1 4 1 1 4 1 1 1 1 1 1 1 0 0 0 0 2 0 1 1 1 1 1 1 1 1 4 1 3 1 4 1
 1 4 1]

CENTROIDES : [19, 24, 0, 2, 9]
